<a href="https://colab.research.google.com/github/lukegwoolley/CodeAcademy/blob/main/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will create a book recommendation algorithm using **K-Nearest Neighbors**.

You will use the [Book-Crossings dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/). This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users. 

After importing and cleaning the data, use `NearestNeighbors` from `sklearn.neighbors` to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures distance to determine the “closeness” of instances.

Create a function named `get_recommends` that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

`get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")`

should return:

```
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301], 
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]
```

Notice that the data returned from `get_recommends()` is a list. The first element in the list is the book title passed in to the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed in to the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.

In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2021-08-24 11:47:16--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  --.-KB/s    in 0.1s    

2021-08-24 11:47:16 (172 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [46]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [47]:
# add your code here - consider creating a new cell for each section of code
#remove users and books with a low amount of ratings as not statistically significant 
counts1 = df_ratings['user'].value_counts()
df_ratings = df_ratings[df_ratings['user'].isin(counts1[counts1 >= 200].index)]
counts = df_ratings['isbn'].value_counts()
df_ratings = df_ratings[df_ratings['isbn'].isin(counts[counts >= 100].index)]

#

combine_book_rating = pd.merge(df_ratings, df_books, on='isbn')
combine_book_rating = combine_book_rating.dropna(axis = 0, subset = ['title'])
combine_book_rating.isnull().values.any()

book_ratingCount = (combine_book_rating.
     groupby(by = ['title'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'totalrating'})
     [['title', 'totalrating']]
    )
book_ratingCount.head()

,title,totalrating
0,1st to Die: A Novel,127
1,A Is for Alibi (Kinsey Millhone Mysteries (Pap...,140
2,A Map of the World,118
3,A Painted House,163
4,A Prayer for Owen Meany,137


In [48]:
rating_with_totalRatingCount = combine_book_rating.merge(book_ratingCount, left_on = 'title', right_on = 'title', how = 'left')
rating_popular_book.head()

,user,isbn,rating,title,author,totalrating
0,277427,002542730X,10.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,82
1,3363,002542730X,0.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,82
2,11676,002542730X,6.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,82
3,12538,002542730X,10.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,82
4,13552,002542730X,0.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,82


In [49]:
book = "Where the Heart Is (Oprah's Book Club (Paperback))"
print(rating_popular_book[rating_popular_book['title']==book])

         user        isbn  ...        author totalrating
31480  278418  0446672211  ...  Billie Letts         183
31481    3363  0446672211  ...  Billie Letts         183
31482    6251  0446672211  ...  Billie Letts         183
31483    6563  0446672211  ...  Billie Letts         183
31484    6575  0446672211  ...  Billie Letts         183
...       ...         ...  ...           ...         ...
31658  266226  0446672211  ...  Billie Letts         183
31659  268932  0446672211  ...  Billie Letts         183
31660  270713  0446672211  ...  Billie Letts         183
31661  271448  0446672211  ...  Billie Letts         183
31662  274808  0446672211  ...  Billie Letts         183

[183 rows x 6 columns]


In [50]:
rating_popular_book = rating_popular_book.drop_duplicates(['user', 'title'])
print(rating_popular_book[rating_popular_book['title']==book])
rating_popular_book_pivot = rating_popular_book.pivot(index = 'title', columns = 'user', values = 'rating').fillna(0)
rating_popular_book_matrix = csr_matrix(rating_popular_book_pivot.values)

model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute', p=2)
model_knn.fit(rating_popular_book_matrix)
print(rating_popular_book_pivot.iloc[rating_popular_book_pivot.index.str.contains(book, regex=False),:])

         user        isbn  ...        author totalrating
31480  278418  0446672211  ...  Billie Letts         183
31481    3363  0446672211  ...  Billie Letts         183
31482    6251  0446672211  ...  Billie Letts         183
31483    6563  0446672211  ...  Billie Letts         183
31484    6575  0446672211  ...  Billie Letts         183
...       ...         ...  ...           ...         ...
31658  266226  0446672211  ...  Billie Letts         183
31659  268932  0446672211  ...  Billie Letts         183
31660  270713  0446672211  ...  Billie Letts         183
31661  271448  0446672211  ...  Billie Letts         183
31662  274808  0446672211  ...  Billie Letts         183

[183 rows x 6 columns]
user                                                254     ...  278418
title                                                       ...        
Where the Heart Is (Oprah's Book Club (Paperback))     0.0  ...     0.0

[1 rows x 894 columns]


In [51]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  distances, indices = model_knn.kneighbors(rating_popular_book_pivot.iloc[rating_popular_book_pivot.index.str.contains(book, regex=False),:].values.reshape(1,-1), n_neighbors = 6)
  recommended_books = []
  rec_dist = []
  for i in range(0, len(distances.flatten())):
      if i == 0:
         print('Recommendations for {0}:\n'.format(book))
         recommended_books.append(book)
      else:
         print('{0}: {1}, with distance of {2}:'.format(i, rating_popular_book_pivot.index[indices.flatten()[i]], distances.flatten()[i]))
         recommended_books.append(rating_popular_book_pivot.index[indices.flatten()[i]])
         rec_dist.append(distances.flatten()[i])

  return recommended_books,rec_dist

In [39]:
book = "The Queen of the Damned (Vampire Chronicles (Paperback))"
distances, indices = model_knn.kneighbors(rating_popular_book_pivot.iloc[rating_popular_book_pivot.index.str.contains(book, regex=False),:].values.reshape(1,-1), n_neighbors = 6)
recommended_books = []
for i in range(0, len(distances.flatten())):
  if i == 0:
      print('Recommendations for {0}:\n'.format(book))
      recommended_books.append(book)
  else:
      print('{0}: {1}, with distance of {2}:'.format(i, rating_popular_book_pivot.index[indices.flatten()[i]], distances.flatten()[i]))
      recommended_books.append(rating_popular_book_pivot.index[indices.flatten()[i]])

Recommendations for The Queen of the Damned (Vampire Chronicles (Paperback)):

1: The Vampire Lestat (Vampire Chronicles, Book II), with distance of 0.5178411602973938:
2: The Tale of the Body Thief (Vampire Chronicles (Paperback)), with distance of 0.5376338362693787:
3: Interview with the Vampire, with distance of 0.7383682131767273:
4: The Witching Hour (Lives of the Mayfair Witches), with distance of 0.7435927391052246:
5: Wizard and Glass (The Dark Tower, Book 4), with distance of 0.7591226100921631:


Use the cell below to test your function. The `test_book_recommendation()` function will inform you if you passed the challenge or need to keep trying.

In [52]:
#books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")

def test_book_recommendation():
  test_pass = True
  recommends, distances = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(4): 
    if recommends[i+1] not in recommended_books:
      test_pass = False
    if abs(distances[i] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

Recommendations for Where the Heart Is (Oprah's Book Club (Paperback)):

1: Blue Diary, with distance of 0.718287467956543:
2: The Lovely Bones: A Novel, with distance of 0.7234864234924316:
3: I Know This Much Is True, with distance of 0.7518420815467834:
4: Icy Sparks, with distance of 0.7731838822364807:
5: Exclusive, with distance of 0.7893686294555664:
You haven't passed yet. Keep trying!
